<a href="https://colab.research.google.com/github/lorenzobusin/7DOS-plugin/blob/Development/hm3_temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Karger algorithm

## Support objects

In [ ]:
!pip install ipython-autotime

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.98 s


In [ ]:
class Graph:
    def __init__(self):
        self.n_nodes, self.n_edges = 0, 0
        self.nodes = defaultdict() # adjacency lists
        self.edges = []

    def addEdge(self, v1:int, v2:int):
        self.nodes[v1].append(v2)
        #self.nodes[v2].append(v1) # Graph is undirected
        if (v2, v1) not in (self.edges):
          self.edges.append((v1, v2))
          self.n_edges += 1

    def buildGraph(self):
        self.n_nodes = len(dataset)
        for i in range (1, len(dataset) + 1):
          self.nodes[i] = []
        for v in dataset:
          for i in range (1, len(v)):
            self.addEdge(v[0], v[i])
                

time: 10.6 ms


## Reading the input

In [ ]:
from collections import defaultdict
def readInput(path: str):
    lines = open(path, "r").read().split("\n")
    input = []
    for line in range(len(lines)-1):
        input.append(list(map(int, lines[line].split()))) # Transform list of strings to list of integers

    return input

time: 4.12 ms


## Karger implementation

In [ ]:
import random
import math

def fullContraction(graph):
    random.seed()
    while graph.n_nodes > 2:
        #print("n_nodes =", graph.n_nodes)
        #print("n_edges =", graph.n_edges)

        e = graph.edges[random.randrange(1, graph.n_edges)]
        #print("e: " , e)
        # Contraction
        id = "Z_" + str(e[0]) + str(e[1])
        graph.nodes[id] = []

        del_edges = 0
        delete_list = []
        for edge in graph.edges: # delete all incident edges
          if edge[0] == e[0] or edge[0] == e[1] or edge[1] == e[0] or edge[1] == e[1]:
            delete_list.append(edge)
            del_edges += 1

        for edge in delete_list:
          graph.edges.remove(edge)

        add_edges = 0
        for v in graph.nodes[e[0]]:
          if v != e[1]:
            graph.addEdge(id, v)
            #print("before:" , graph.nodes[v])
            graph.nodes[v].remove(e[0])
            #print("deleting: ", e[0])
            #print("after: ", graph.nodes[v])
            graph.nodes[v].append(id)

        for v in graph.nodes[e[1]]:
          if v != e[0]:
            graph.nodes[id].append(v)
            graph.edges.append((id, v))
            add_edges += 1
            graph.nodes[v].append(id)
            graph.nodes[v].remove(e[1])
              

        graph.nodes[e[0]] = []
        graph.nodes[e[1]] = []

        graph.n_nodes -= 1
        graph.n_edges -= (del_edges - add_edges)  # Need to compute molteplicity

    return graph.n_edges, graph

def karger(graph, k):
    min = math.inf
    for i in range(k):
        t, g = fullContraction(graph)
        print(t)
        if t < min: min = t
    return min, g

time: 38.2 ms


In [ ]:
dataset = readInput("input_random_13_50.txt")
print(dataset)

graph = Graph()
graph.buildGraph()
print(graph.nodes)
print(graph.edges)

k = 10
random.seed(random.randrange(1, 1000))
result, g = karger(graph, k)
print("final result =", result)

[[1, 36, 7, 35, 12, 42, 10, 30, 48, 37, 31, 21, 18, 25, 49, 20, 23, 15, 4, 5, 6, 8, 9, 14, 27, 32, 34, 39, 41, 44, 45, 50], [2, 46, 35, 25, 49, 5, 6, 9, 15, 20, 21, 31, 34, 36, 40, 42, 43, 44, 45, 47, 48], [3, 33, 16, 4, 7, 12, 21, 22, 23, 27, 28, 30, 35, 37, 42, 49], [4, 22, 5, 39, 42, 18, 37, 3, 19, 11, 16, 47, 12, 6, 50, 1, 14, 23, 27, 32, 44, 45, 46, 25, 9, 10, 20, 30, 33, 35, 38, 49], [5, 4, 19, 1, 13, 37, 46, 11, 2, 20, 31, 16, 47, 22, 9, 21, 7, 8, 23, 24, 25, 26, 28, 29, 30, 33, 34, 39, 41, 42, 48, 49], [6, 4, 2, 43, 31, 24, 20, 47, 1, 16, 36, 7, 8, 9, 13, 19, 23, 25, 30, 34, 35, 41, 42, 44, 45, 49], [7, 1, 48, 38, 35, 11, 8, 43, 25, 3, 16, 27, 45, 30, 41, 12, 15, 34, 22, 28, 5, 21, 46, 49, 6, 39, 17, 19, 20, 23, 26, 33, 37, 40, 42], [8, 7, 6, 49, 23, 42, 19, 16, 41, 5, 1, 32, 12, 20, 25, 26, 29, 30, 31, 34, 36, 37, 39, 48], [9, 5, 17, 31, 36, 2, 10, 30, 6, 19, 48, 47, 4, 21, 34, 1, 20, 25, 23, 42, 12, 13, 26, 29, 32, 41, 43, 49], [10, 1, 9, 28, 47, 15, 25, 37, 33, 4, 48, 45, 12